# ① 市町村別気象データを都道府県気象データに加工する

# モジュールのインポート

In [ ]:
import glob
import pandas as pd
import numpy as np

関数

In [ ]:
def nan_check():
  for t_df in df_list:
    print(t_df.head())
    # 各列のNaNの確認
    print(t_df['temp'].hasnans)
    print(t_df['prec'].hasnans)
    print(t_df['sun'].hasnans)
    print(t_df['snow'].hasnans)
    print(t_df['humi'].hasnans)

#ファイルの取り込み
1. Numbersで処理したcsvを都道府県ごとにcolab上にアップロード

2. 市町村全てのデータを取り込む

愛知県のデータ例: aa_22_m.csv, ag_22_m.csv, ai_22_m.csv, am_22_m.csv, ao_22_m.csv, as_22_m.csv, at_22_m.csv, aty_22_m.csv

In [ ]:
file_list = glob.glob('/content/*.csv')
print(type(file_list))
print(file_list)

#ファイルの欠損確認
1. pandasをインポート

2. 市町村データファイルをデータフレームに変換する

3. データを確認する

4. データのinfoを確認する

5. 各気象データの品質の数を確認する

6. 各気象データのNaNを確認する

In [ ]:
import pandas as pd

for f in file_list:
  df= pd.read_csv(f) # 個別にファイル名を指定
  print(df.head())
  df.info()
  print(df['temp_q'].value_counts())
  print(df['prec_q'].value_counts())
  print(df['sun_q'].value_counts())
  print(df['snow_q'].value_counts())
  print(df['humi_q'].value_counts())

  # 各列のNaNの確認
  print(df['temp'].hasnans)
  print(df['prec'].hasnans)
  print(df['sun'].hasnans)
  print(df['snow'].hasnans)
  print(df['humi'].hasnans)


（オプション）品質情報1のカラムのデータ確認

※ 品質情報1:欠損、品質情報4:データとしての扱いに注意、品質情報5:データとして使用可能、品質情報8:正常データ

In [ ]:
df.query('temp_q == 1')

#欠損データを埋める

欠損データの処理
気温データの場合例：

データdf['temp']がNaNかつ品質番号df['temp_q']が1である時、前後データから予測して欠損値を埋める。

それ以外の場合のNaNは、0で埋める

欠損値処理したデータからデータフレームを作成

データフレームをリスト化する

In [ ]:
import pandas as pd
df_list = []
for f in file_list:
  df= pd.read_csv(f) # 個別にファイル名を指定
  if df['temp'].hasnans and 1 in df['temp_q'].values:
    df['temp'].interpolate(inplace= True, limit_direction='both')
  else:
    df['temp'].fillna(0,inplace = True)

  if df['prec'].hasnans and 1 in df['prec_q'].values:
    df['prec'].interpolate(inplace= True, limit_direction='both')
  else:
    df['prec'].fillna(0,inplace = True)

  if df['sun'].hasnans and 1 in df['sun_q'].values:
    df['sun'].interpolate(inplace= True, limit_direction='both')
  else:
    df['sun'].fillna(0,inplace = True)

  if df['snow'].hasnans and 1 in df['snow_q'].values:
    df['snow'].interpolate(inplace= True, limit_direction='both')
  else:
    df['snow'].fillna(0,inplace = True)

  if df['humi'].hasnans and 1 in df['humi_q'].values:
    df['humi'].interpolate(inplace= True, limit_direction='both')
  else:
    df['humi'].fillna(0,inplace = True)

 #削除ではなく抽出
  t_df = pd.DataFrame(df, columns= ['temp', 'prec', 'sun', 'snow', 'humi'])

  # df_listに追加する。
  df_list.append(t_df)

df_list

（オプション）欠損値データ確認

In [ ]:
nan_check()

#数値を平均してデータフレームを結合する

欠損処理した市町村データを気象条件ごとに平均値および標準偏差を算出し1つの都道府県データとする。

In [ ]:
df_m = pd.DataFrame(np.mean([df_list[0], df_list[1], df_list[2], df_list[3], df_list[4], df_list[5], df_list[6], df_list[7]], 0), columns=t_df.columns)
df_s = pd.DataFrame(np.std([df_list[0], df_list[1], df_list[2], df_list[3], df_list[4], df_list[5], df_list[6], df_list[7]], 0), columns=t_df.columns)


#平均の調節
df_m['snow'] = df_m['snow'] *8/4
df_m['humi'] = df_m['humi'] *8/1

rename_std()

all_df = pd.merge(df[['date']], df_m, left_index=True, right_index= True)

pref_df = pd.merge(all_df, df_s[['temp_s', 'prec_s', 'sun_s', 'snow_s', 'humi_s']], left_index=True, right_index= True)

pref_df.head()

#市町村データから都道府県データにまとめたものをcsv形式で保存する

In [ ]:
pref_df.to_csv('pref_df.csv')

# ② 気象データにブロッコリー収穫量データを結合する

# モジュールのインポート

In [ ]:
mport pandas as pd

ブロッコリー収穫量をデータフレームに結合し、気象データごとに分割して保存する

In [ ]:
# 収穫量をデータフレームに追加して気象データごとにファイルを分割して保存する

# csvファイルを取り込む
df = pd.read_csv('/content/pref_df.csv')

# ブロッコリー収穫量をカラムに追加する
df['Broc']= 1410

# 気温データのデータフレームを作成して、保存
Br_temp = df[['date','temp','Broc']]
Br_temp.to_csv('tBroc_pref_df.csv', index=False)

# 降水量データのデータフレームを作成して、保存
Br_prec = df[['date', 'prec', 'Broc']]
Br_prec.to_csv('pBroc_pref_df.csv', index=False)

# 日照時間データのデータフレームを作成して、保存
Br_sun = df[['date', 'sun', 'Broc']]
Br_sun.to_csv('sBroc_pref_df.csv', index=False)

# 降雪量データのデータフレームを作成して、保存
Br_snow = df[['date', 'snow', 'Broc']]
Br_snow.to_csv('snBroc_pref_df.csv', index=False)

# 湿度データのデータフレームを作成して、保存
Br_humi = df[['date', 'humi', 'Broc']]
Br_humi.to_csv('hBroc_pref_df.csv', index=False)

# ブロッコリーデータをpref_dfに加えて、保存
df.to_csv('pref_df.csv', index=False)

# 線形回帰による収穫量の予測

1. モジュールをインポートする。

2. csvファイルからデータセットのリストを作る

3. 学習データとテストデータを分割する

4. モデルの評価

In [ ]:
import pandas as pd
import numpy as np
import glob
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

#ファイルを読み込む
file_list = glob.glob('/content/*.csv')
print(file_list) #取り込んだファイルの順序を表示する

Broc = []
data_list = []
for file in file_list:
  df = pd.read_csv(file)
  Broc.append(df['Broc'][0]) # ブロッコリー収穫量をリスト化する　Xにあたる

  df = df[['temp','prec', 'sun', 'snow', 'humi']] # 気象データ5条件を取り出す
  data_list.append(df.values.ravel()) # (365, 5)データを一次元配列にして、読み込みファイル分リスト化する　yにあたる

# 線形で回帰する
X_train, X_test, y_train, y_test = train_test_split(data_list, Broc, test_size= 0.3, random_state=0)
model = LinearRegression()
model.fit(X_train, y_train)
model.score(X_test,y_test)
model.predict(X_test)

# ランダムフォレスト回帰による収穫量の予測

1. モジュールをインポートする。

2. csvファイルからデータセットのリストを作る

3. 学習データとテストデータを分割する

4. モデルの評価

In [ ]:
import pandas as pd
import numpy as np
import glob
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

#ファイルを読み込む
file_list = glob.glob('/content/*.csv')
print(file_list) #取り込んだファイルの順序を表示する

Broc = []
data_list = []
for file in file_list:
  df = pd.read_csv(file)
  Broc.append(df['Broc'][0]) # ブロッコリー収穫量をリスト化する　Xにあたる

  df = df[['temp','prec', 'sun', 'snow', 'humi']] # 気象データ5条件を取り出す
  data_list.append(df.values.ravel()) # (365, 5)データを一次元配列にして、読み込みファイル分リスト化する　yにあたる

# ランダムフォレストで回帰する場合
X_train, X_test, y_train, y_test = train_test_split(data_list, Broc, test_size= 0.3, random_state=0)
model = RandomForestRegressor(n_estimators=10, criterion='squared_error', max_depth=None)
model.fit(X_train, y_train)
model.score(X_test,y_test)
model.predict(X_test)

# 収穫量をクラスタリングし、予測データに用いる
都道府県ファイルを読み込む

各気象条件ごとにデータをリスト化する


In [ ]:
#全てのファイルを読み込む
file_list = glob.glob('/content/*.csv')

# 正規表現を使って、気象データごとにファイルをリスト化する
import re

# 気温データ19道府県をリスト化する
file_name_pattern = re.compile(r'.*tBroc.*')
file_t =[]
for file_name in file_list:
  # ファイル名に対する正規表現パターンが一致するか確認
  if file_name_pattern.match(file_name):
    file_t.append(file_name)
  else:
    continue

# 日照時間データ19道府県をリスト化する
file_name_pattern = re.compile(r'.*sBroc.*')
file_s =[]
for file_name in file_list:
  # ファイル名に対する正規表現パターンが一致するか確認
  if file_name_pattern.match(file_name):
    file_s.append(file_name)
  else:
    continue

# 降水量データ19道府県をリスト化する
file_name_pattern = re.compile(r'.*pBroc.*')
file_p =[]
for file_name in file_list:
  # ファイル名に対する正規表現パターンが一致するか確認
  if file_name_pattern.match(file_name):
    file_p.append(file_name)
  else:
    continue

# 降雪量データ19道府県をリスト化する
file_name_pattern = re.compile(r'.*snBroc.*')
file_sn =[]
for file_name in file_list:
  # ファイル名に対する正規表現パターンが一致するか確認
  if file_name_pattern.match(file_name):
    file_sn.append(file_name)
  else:
    continue

# 湿度データ19道府県をリスト化する
file_name_pattern = re.compile(r'.*hBroc.*')
file_h =[]
for file_name in file_list:
  # ファイル名に対する正規表現パターンが一致するか確認
  if file_name_pattern.match(file_name):
    file_h.append(file_name)
  else:
    continue

各気象条件データをクラスタリングする


In [ ]:
# 湿度

# リストにデータを追加する
Broc = []
Humi = []

for f in file_h:
  df= pd.read_csv(f) # 個別にファイル名を指定

  df = df.T #行列を入れ替える

  df.columns = df.iloc[0] #日付をカラムにする

  df = df.drop(df.index[0]) #日付行を削除する

  # リストにデータを追加する
  Broc.append(df.loc['Broc'][0]) #BrocリストにBrocの値を追加

  Humi.append(df.loc['humi'].values) # TempリストにTempの値を追加

# KMeansでクラスタリング
model = KMeans(n_clusters=4, random_state=0)
model.fit(Humi)

df_humi = pd.DataFrame(data = Humi)

df_humi['クラス'] = model.labels_
df_m['humi_c']=df_humi['クラス']
import re

# 正規表現パターン
pattern = re.compile(r'_([A-Z]+)_')
pref_list = []
for f in file_h:
  match = pattern.search(f) # ファイル名から正規表現に一致する部分を検索
  if match: # 一致した部分を取り出す
    result = match.group(1)
    pref_list.append(result)
  else:
    print('Match not found.')

df_humi['pref'] = pref_list
df_humi['humi_c']=df_humi['クラス']

# 日照時間

# リストにデータを追加する
Broc = []
Sun = []

for f in file_s:
  df= pd.read_csv(f) # 個別にファイル名を指定

  df = df.T #行列を入れ替える

  df.columns = df.iloc[0] #日付をカラムにする

  df = df.drop(df.index[0]) #日付行を削除する

  # リストにデータを追加する
  Broc.append(df.loc['Broc'][0]) #BrocリストにBrocの値を追加

  Sun.append(df.loc['sun'].values) # TempリストにTempの値を追加


# KMeansでクラスタリング
model = KMeans(n_clusters=4, random_state=0)
model.fit(Sun)

df_sun = pd.DataFrame(data = Sun)

df_sun['クラス'] = model.labels_
df_m['sun_c']=df_sun['クラス']

import re

# 正規表現パターン
pattern = re.compile(r'_([A-Z]+)_')
pref_list = []
for f in file_s:
  match = pattern.search(f) # ファイル名から正規表現に一致する部分を検索
  if match: # 一致した部分を取り出す
    result = match.group(1)
    pref_list.append(result)
  else:
    print('Match not found.')

df_sun['pref'] = pref_list
df_sun['sun_c']=df_prec['クラス']


#　降水量

# リストにデータを追加する
Broc = []
Prec = []

for f in file_p:
  df= pd.read_csv(f) # 個別にファイル名を指定

  df = df.T #行列を入れ替える

  df.columns = df.iloc[0] #日付をカラムにする

  df = df.drop(df.index[0]) #日付行を削除する

  # リストにデータを追加する
  Broc.append(df.loc['Broc'][0]) #BrocリストにBrocの値を追加

  Prec.append(df.loc['prec'].values) # PrecリストにPrecの値を追加

#KMeansでクラスタリング　クラスタ4
model = KMeans(n_clusters=4, random_state=0)
model.fit(Prec)

df_prec = pd.DataFrame(data = Prec)

df_prec['クラス'] = model.labels_
df_m['prec_c']=df_prec['クラス']

import re

# 正規表現パターン
pattern = re.compile(r'_([A-Z]+)_')
pref_list = []
for f in file_p:
  match = pattern.search(f) # ファイル名から正規表現に一致する部分を検索
  if match: # 一致した部分を取り出す
    result = match.group(1)
    pref_list.append(result)
  else:
    print('Match not found.')

df_prec['pref'] = pref_list
df_prec['prec_c']=df_prec['クラス']


#雪

# リストにデータを追加する
Broc = []
Snow = []

for f in file_sn:
  df= pd.read_csv(f) # 個別にファイル名を指定

  df = df.T #行列を入れ替える

  df.columns = df.iloc[0] #日付をカラムにする

  df = df.drop(df.index[0]) #日付行を削除する

  # リストにデータを追加する
  Broc.append(df.loc['Broc'][0]) #BrocリストにBrocの値を追加

  Snow.append(df.loc['snow'].values) # TempリストにTempの値を追加


#KMeansでクラスタリング クラスタ4
model = KMeans(n_clusters=4, random_state=0)
model.fit(Snow)

df_snow = pd.DataFrame(data = Snow)

df_snow['クラス'] = model.labels_
df_m['snow_c']=df_snow['クラス']

Date = df.columns.tolist()


import re

# 正規表現パターン
pattern = re.compile(r'_([A-Z]+)_')
pref_list = []
for f in file_sn:
  match = pattern.search(f) # ファイル名から正規表現に一致する部分を検索
  if match: # 一致した部分を取り出す
    result = match.group(1)
    pref_list.append(result)
  else:
    print('Match not found.')

df_snow['pref'] = pref_list
df_snow['snow_c']=df_snow['クラス']


#　気温

# リストにデータを追加する
Broc = []
Temp = []

for f in file_t:
  df= pd.read_csv(f) # 個別にファイル名を指定

  df = df.T #行列を入れ替える

  df.columns = df.iloc[0] #日付をカラムにする

  df = df.drop(df.index[0]) #日付行を削除する

  # リストにデータを追加する
  Broc.append(df.loc['Broc'][0]) #BrocリストにBrocの値を追加

  Temp.append(df.loc['temp'].values) # TempリストにTempの値を追加

#KMeansでクラスタリング
model = KMeans(n_clusters=4, random_state=0)
model.fit(Temp)

df_temp = pd.DataFrame(data = Temp)

df_temp['クラス'] = model.labels_
df_m['temp_c']=df_temp['クラス']


import re

# 正規表現パターン
pattern = re.compile(r'_([A-Z]+)_')
pref_list = []
for f in file_t:
  match = pattern.search(f) # ファイル名から正規表現に一致する部分を検索
  if match: # 一致した部分を取り出す
    result = match.group(1)
    pref_list.append(result)
  else:
    print('Match not found.')

df_temp['pref'] = pref_list
df_temp['temp_c']=df_temp['クラス']
df_temp['Broc'] = Broc


import pandas as pd

# カラムAとBを結合する
result_df = pd.merge(df_temp[['temp_c', 'pref','Broc']],
                     df_snow[['snow_c', 'pref']],
                     on='pref')

result_df = pd.merge(result_df,
                     df_humi[['humi_c', 'pref']],
                     on='pref')

result_df = pd.merge(result_df,
                     df_sun[['sun_c', 'pref']],
                     on='pref')

result_df = pd.merge(result_df,
                     df_prec[['prec_c', 'pref']],
                     on='pref')


result_df
